In [73]:
#        ,----,                                          
#      ,/   .`|                                          
#    ,`   .'  :                      ,---,               
#  ;    ;     /                    .'  .' `\             
#.'___,/    ,'  ,---.      ,---,.,---.'     \    ,---.   
#|    :     |  '   ,'\   ,'  .' ||   |  .`\  |  '   ,'\  
#;    |.';  ; /   /   |,---.'   ,:   : |  '  | /   /   | 
#`----'  |  |.   ; ,. :|   |    ||   ' '  ;  :.   ; ,. : 
#    '   :  ;'   | |: ::   :  .' '   | ;  .  |'   | |: : 
#    |   |  ''   | .; ::   |.'   |   | :  |  ''   | .; : 
#    '   :  ||   :    |`---'     '   : | /  ; |   :    | 
#    ;   |.'  \   \  /           |   | '` ,/   \   \  /  
#    '---'     `----'            ;   :  .'      `----'   
#                                |   ,.'                 
#                                '---'                   
#                                                        

# COMO ME LLEGA EL CORPUS? El corpus lo leo ya sea de un txt o de un csv
# COMO Y POR Q SEPARARLO: Las mananeras vienen secuencialmente y hay periodos donde se repte, se puede separar por fechas y randomizar las oraciones dentro.
# QUE ES UNA SEMILLA DE REPRODUCIBILIDAD: 
# Hacer un corpus de prueba para las siguientes ...
# 1) Separar el corpus  el conjuntos entrenamiento y prueba . 
# Recordar que el conjunto que prueba debe ser creado de manera aleatoria, por lo tanto, 
# debe de fijarse una semilla para lograr reproducibilidad.

# COMO ANALIZAR UN CORPUS CON GRAFICAS BONITAS ...
# 2) Hacer un análisis exploratorio del conjunto de entrenamiento para decidir sobre el siguiente paso. 
# Una guía es la siguiente: https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools

# ---------------------------------------
# QUE DECISIONES VOY A TOMAR EN MI CORPUS
# 3) En función del análisis previo: limpiar, normalizar, segmentar y tokenizar el corpus de entrenamiento.  
# 4) Las palabras poco frecuentes transformarlas en el token <UNK>. 

# CORPUS LISTO PARA EL MODELO DE N-GRAMAS
# CUAL ES MI OUTPUT, QUE NECESITA?

# 5) Una vez armado el corpus de entrenamiento, construir el corpus ordenado, listo para ser usado en el modelo de n-gramas,
#    debe tener la estructura [['token','token', ...'token'],['token','token', ...'token'] .. ['token','token', ...'token']]. 

# PUEDO AGREGAR OTRAS TECNICAS AL VOCAB?
# 6) Crea el vocabulario

# **Obteniendo el corpus** 📖

In [74]:
def getData(filename: str) -> str:
    
    rawData = ""
    with open(filename, encoding="utf-8") as f:
        for line in f:
            rawData += line
    return rawData

In [75]:
import os

filepath = os.path.join(os.path.dirname(os.getcwd()), "data", "corpus.txt")

rawCorpus = getData(filepath)
print(f"Carácteres: {len(rawCorpus)}")
print(f"Extracto: \n{rawCorpus[:500]}")

Carácteres: 30562044
Extracto: 
 Buenas tardes.
Sí. Es que son buenos días, pero llegué tarde, porque estábamos tratando temas importantes, desde luego, el tema de la seguridad.
Y hoy nos acompañó la jefa de gobierno de la Ciudad de México, estuvo en la reunión del gabinete de seguridad. Y también se atendió el Programa de Protección a Peregrinos, porque ya estamos en la víspera del día 12 y vienen peregrinos del país, por las carreteras. Y se habló del programa de protección.
Lo mismo del programa de protección, de apoyo, de 


## **Análisis exploratorio** 🔎

El propósito es realizar un análisis exploratorio sobre los datos crudos para tener una base en la toma de desiciones con respecto al pre-procesamiento de datos (limpieza, normalización, etc.).

### Estadísticas simples

- Cantidad de palabras (secuencias de caractéres separados por espacios).
- Cantidad de oraciones (secuencias de palabras separados por un punto).

In [76]:
from nltk.tokenize import word_tokenize, sent_tokenize
words=word_tokenize(rawCorpus, "spanish")
sentences=sent_tokenize(rawCorpus, "spanish")

print("Cantidad de carácteres:", len(rawCorpus))
print("Cantidad de palabras:", len(words))
print("Cantidad de oraciones:", len(sentences))
print("Promedio de carácteres por palabra:", round(len(rawCorpus) / len(words)))
print("Promedio de palabras por oración:", round(len(words) / len(sentences)))

### Distribución del tamaño de las palabras

In [6]:
import plotly.express as pt

fig = pt.violin([len(word) for word in words], box=True, title="Distribución del tamaño de las palabras")
fig.update_layout(yaxis_title="Tamaño de palabra")
fig.update_layout(xaxis_title="")
fig.show()

Quitamos los signos de punctuación para observar como cambia la distribución.

In [7]:
import plotly.express as pt
import re

def stripWord(word: str) -> str:
    """ Elimina los signos de punctuación de una palabra. """
    return re.sub('\W+','', word)

fig = pt.violin([len(word) for word in map(stripWord, words) if word], box=True, 
                title="Distribución del tamaño de las palabras sin sígnos de punctuación.")
fig.update_layout(yaxis_title="Tamaño de palabra")
fig.update_layout(xaxis_title="")
fig.show()

### Distribución del tamaño de las oraciones

In [8]:
fig = pt.violin([len(word_tokenize(sentence, "spanish")) for sentence in sentences], box=True, title="Distribución del tamaño de las oraciones")
fig.update_layout(yaxis_title="Tamaño de oración")
fig.update_layout(xaxis_title="")
fig.show()

## **Limpieza de datos** 🧹

In [9]:
def cleanWord(word: str) -> str:
    """ Elimina los signos de punctuación de una palabra excepto el punto y coma. """
    return re.sub(r'[^\w\,\.]','', word)

def cleanSentence(sentence: str) -> list:
    """ Regresa un listado de las palabras límpias de una oración. """
    words = word_tokenize(sentence)
    cleanWords = [word for word in map(cleanWord, words) if word]
    return cleanWords

In [10]:
cleanSentences = [cleanSentence(sentence) for sentence in sentences]
nCleanWords = sum(len(word) for word in cleanSentences)
print(f"Se limpiaron {len(words) - nCleanWords} palabras.")
print("Promedio de palabras por oración aplicando limpieza:", round( nCleanWords/ len(cleanSentences)))

Se limpiaron 262 palabras.
Promedio de palabras por oración aplicando limpieza: 19


### Análizando las palabras en el corpus

In [11]:
from collections import defaultdict, Counter
dic=defaultdict(int)

for sentence in cleanSentences:
    for word in sentence:
        dic[word] += 1
        
wordFreq = Counter(dic)
mostFreq = wordFreq.most_common()

In [12]:
import pandas as pd
df = pd.DataFrame(mostFreq[:10])
df.columns = ["palabra", "frecuencia"]
fig = pt.bar(df, x="palabra", y="frecuencia", color="palabra", title="Frecuencia de palabras")
fig.show()

### Sin contar las 'stopwords'

In [13]:
from nltk.corpus import stopwords
stop=set(stopwords.words('spanish'))
stop.add(".")
stop.add(",")

# Filtering stop words from dic
newDic = dict(filter(lambda pair: pair[0] not in stop, dic.items()))
wordFreq = Counter(newDic)
mostFreq = wordFreq.most_common()

df = pd.DataFrame(mostFreq[:10])
df.columns = ["palabra", "frecuencia"]
fig = pt.bar(df, x="palabra", y="frecuencia", color="palabra", title="Frecuencia de palabras sin contar stop-words")
fig.show()

### Wordcloud de palábras

In [14]:
#%pip install wordcloud

In [15]:
from wordcloud import WordCloud, STOPWORDS

cleanText = " ".join(word for sentence in cleanSentences for word in sentence if word not in stop)

word_cloud = WordCloud(
    width=3000,
    height=2000,
    random_state=1,
    background_color="white",
    colormap="jet_r",
    collocations=False,
    stopwords=STOPWORDS,
    max_words=30
    ).generate(cleanText)

In [16]:
fig = pt.imshow(word_cloud)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

## Filtrando palabras con poca ocurrencia

In [17]:
fig = pt.violin(dic.values(), box=True, title="Distribución de ocurrencia de palabras")
fig.update_layout(yaxis_title="Veces que aparece")
fig.update_layout(xaxis_title="")
fig.show()

Definiremos a una palabra como poco frecuente si está alejada 4 desviaciones estandar de la media por debajo

In [18]:
N = len(dic)
median = sum(dic.values()) / N
desvStd = sum((value-median)**2 for value in dic.values()) ** 0.5 / N
finalVocab = dict()
counter = 0
for word, freq in dic.items():
    if freq > median - 4 * desvStd:
        finalVocab[word] = freq
    else:
        counter += 1

In [19]:
sizeCorpus = sum(dic.values())
print(f"""Una palabra será <UKN> si aparece menos de {min(finalVocab.values())} veces en el corpus.
Se generaron {counter} OOV de {sizeCorpus} palabras, un {counter/sizeCorpus:.2f}%.""")

Una palabra será <UKN> si aparece menos de 4 veces en el corpus.
Se generaron 3244 OOV de 36085 palabras, un 0.09%.


### Corpus final

In [20]:
finalCorpus = []
for sentence in cleanSentences:
    finalSentence = [word if word in dic else "<UKN>" for word in sentence]
    finalCorpus.append(finalSentence)

In [58]:
def saveCorpus(corpus: list[list], path: str) -> None:
    """ Guarda un corpus generado al correr esta libreta en un archivo txt definido en path """
    with open(path, mode='a', encoding='utf-8') as f:
        for sentence in corpus:
            f.write(" ".join(word for word in sentence) + '\n')
    print("done.")

def getCorpus(path: str) -> list[list]:
    """ Recupera un corpus guardado en un archivo de txt """
    corpus = []
    with open(path, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            corpus.append(line.split())
    print("done.")
    return corpus

In [59]:
datasetPath = os.path.join(os.path.dirname(os.getcwd()), "data")
outpath = os.path.join(datasetPath, "cleanExtract.txt")
saveCorpus(finalCorpus, outpath)

done.


## **Conjunto de entrenamiento y conjunto de prueba**

In [71]:
import random
random.seed(2023)
def splitCorpus(corpus: list[list], splitFactor=0.2) -> tuple:
    """" Separa los datos en un conjunto de entenamiento y uno de prueba, 
         de tal forma que si splitFactor = 0.2 entonces un 20% estará en 
         el conjunto de prueba y el restante el de entrenamiento. """
    splitFactor = max(0.2, splitFactor)
    N = len(corpus)

    randomCorpus = corpus.copy()
    random.shuffle(randomCorpus)
    trainCorpus = randomCorpus[int((N+1)*splitFactor):]
    testCorpus = randomCorpus[:int((N+1)*splitFactor)]

    return trainCorpus, testCorpus

In [72]:
datasetPath = os.path.join(os.path.dirname(os.getcwd()), "data")
data = getCorpus(datasetPath + "/cleanExtract.txt")
X_train, X_test = splitCorpus(data)
print(f"{len(X_train)} oraciones en el conjunto de entrenamiento.\n{len(X_test)} en el de pruebas.")

done.
3091 oraciones en el conjunto de entrenamiento.
773 en el de pruebas.
